In [21]:
from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda

import requests
import pandas as pd
import numpy as np
import ast
pd.options.display.max_columns=None

Tendréis que usar el csv attacks_limpieza_completa que tenéis adjunto abajo.
En la lección de hoy aprendimos como transformar nuestros datos para que estén preparados para almacearlos en una BBDD. En este momento tenemos dos fuentes de datos:

1.- El csv con los ataques de tiburones que hemos estado limpiando hasta ahora, el que os hemos adjuntado (attacks_limpieza_completa). Sentiros libres de usar vuestros propios csv en caso de que queráis.

2.- El csv con los datos climáticos de los principales paises que tienen ataques de tiburones, el que creamos en el pair programming de ayer.

El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes. Para ello:

- Cargaremos los dos ficheros de datos

In [2]:
attack=pd.read_csv('../ETL/datos/attacks_limpieza_completa.csv', index_col=0)
df=pd.read_csv('../ETL/datos/df_API_paises.csv', index_col=0)

- Del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer:
    - USA
    - Australia
    - New Zealand
    - South Africa
    - Papua New Guinea


In [3]:
df_attack=attack[(attack["country"]=="usa") | (attack["country"]=="australia")|\
          (attack["country"]=="south africa") | (attack["country"]=="papua new guinea")|\
          (attack["country"]=="new zealand")]

In [4]:
df_attack.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,...,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,...,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,...,1,0,0,1,0,0,3,7,7,3.804860


- Del dataframe de los datos climáticos seleccionaremos todas las columnas.

In [5]:
df.sample(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,paises
42,129,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 150, 'speed':...",24,2,7,1014,none,3,0,160,2,39.78373,-100.445882,USA
17,54,2,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 5}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 5, 'speed': 3...",26,2,5,1010,none,3,0,40,2,39.78373,-100.445882,USA


- Cuando ya tengamos todos los datos deseados juntaremos los dos csv.
    - Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.

In [37]:
#el groupby es por pais

- Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. Si intentamos hacer la media de eso no nos dará un valor real. A este problema ya nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consigierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. Os dejamos el código documentado. ⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.

In [24]:
x = df['rh_profile'].apply(pd.Series)
x[0]=x[0].apply(ast.literal_eval)
x=x[0].apply(pd.Series)


for values in range(len(x.columns)):
    nombre='rh' + str(x[values].apply(pd.Series)["layer"][0])
    valores = list(x[values].apply(pd.Series)['rh'])
    df.insert(values, nombre, valores) 

ValueError: cannot insert rh950mb, already exists

In [22]:
df.head()

,rh950mb,rh900mb,rh850mb,rh800mb,rh750mb,rh700mb,rh650mb,rh600mb,rh550mb,rh500mb,rh450mb,rh400mb,rh350mb,rh300mb,rh250mb,rh200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,paises
0,12,10,10,10,10,9,6,5,10,13,15,16,16,16,16,15,3,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 235, 'speed':...",24,-1,11,1007,rain,2,0,290,2,39.78373,-100.445882,USA
1,8,7,6,9,8,3,1,3,7,13,15,16,16,16,16,15,6,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 8}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 275, 'speed':...",24,-4,10,1008,rain,3,0,300,2,39.78373,-100.445882,USA
2,6,4,4,7,5,1,0,6,11,7,12,16,16,16,16,6,9,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 6}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 225, 'speed':...",26,-4,7,1009,none,3,0,220,3,39.78373,-100.445882,USA
3,4,6,6,4,3,4,6,8,10,5,-1,10,16,16,16,0,12,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 4}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 225, 'speed':...",28,-1,3,1009,none,3,0,210,3,39.78373,-100.445882,USA
4,2,3,5,8,8,7,3,3,8,4,2,3,2,4,3,0,15,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 225, 'speed':...",29,-1,2,1008,none,3,0,185,3,39.78373,-100.445882,USA


In [31]:
y = df['wind_profile'].apply(pd.Series)
y[0]=y[0].apply(ast.literal_eval)
y=y[0].apply(pd.Series)

for values in range(len(y.columns)):
    nombre='wind' + str(y[values].apply(pd.Series)['layer'][0])
    valores = list(y[values].apply(pd.Series)['wind'])
    df.insert(values, nombre, valores) 

KeyError: 'wind'

- Guardar los resultados obtenidos en un csv que usaremos en próximos ejercicios de pair programming.